# [DCC 030] Aprendizado Profundo para Processamento de Linguagem Natural: Trabalho Prático 2
__Aluno:__
- Eduardo Villani de Carvalho Filho - 2015104008
---
# [Parte 1] O trabalho

## Bibliotecas que serão utilizadas

Aqui é definido todas as bibliotecas que serão utilizadas ao longo deste trabalho.

In [44]:
import pathlib
import nltk
import ssl
import numpy as np
import joblib
import tqdm

from typing import Union


## Eventual problema de SSL

Há um pequeno problema no sistema de download da biblioteca utilizada, por isso devemos rodar a
linha abaixo para configurar o certificado ssl do computador.

In [2]:

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

## Introdução

Este trabalho tem como objetivo pôr em prática o que foi visto em sala de aula do conceito de
POS Tagging. O POS Tagging tem por objetivo classificar as palavras em classes gramaticais, isto
é, pegando uma frase genérica como: "Eu lavei meu carro ontem", o POS Tagging teria por objetivo
saber classificar "Eu" como um Pronome Pessoa, "lavei" como um verbo, "meu" como um pronome
possessivo, "carro" como um substantivo e "ontem" como um advérbio de tempo, além de outras
classificações possíveis, como "meu carro ontem" como um objeto direto.

Para a elaboração deste trabalhos, iremos utilizar o corpus Mac Morpho (colocar link aqui),
o qual contém classificação de sentenças em português brasileiro feita pela USP. Pegando um exemplo prático
utilizando a biblioteca do Python NLTK:

In [3]:
nltk.download('mac_morpho')
mac_morpho = nltk.corpus.mac_morpho
print(len(mac_morpho.tagged_sents()))
for word_tagged in mac_morpho.tagged_sents()[0]:
    print(word_tagged)

del word_tagged

[nltk_data] Downloading package mac_morpho to
[nltk_data]     /Users/eduardovillani/nltk_data...
[nltk_data]   Package mac_morpho is already up-to-date!


51397
('Jersei', 'N')
('atinge', 'V')
('média', 'N')
('de', 'PREP')
('Cr$', 'CUR')
('1,4', 'NUM')
('milhão', 'N')
('em', 'PREP|+')
('a', 'ART')
('venda', 'N')
('de', 'PREP|+')
('a', 'ART')
('Pinhal', 'NPROP')
('em', 'PREP')
('São', 'NPROP')
('Paulo', 'NPROP')


Como podemos notar, o nltk apresenta a classificação de cada palavra e ainda a junção de dois tipos
diferentes para formar um só, como de + a formando da, como uma preposição e um artigo.

Ao longo deste trabalho, iremos ver o funcionamento do NLTK (para se familiarizar com a biblioteca e o POS Tagging),
passando por uma avaliação dos classificadores do NLTK e finalizando momontando um modelo usando XYZ
para comparação com estes classificadores.

# [Parte 2] Conhecendo o nltk e o MacMorphus

bla bla bla

In [ ]:
def most_commom_tags(corpus, n):
    def simplify_tag(t):
        """Retirado de: http://www.nltk.org/howto/portuguese_en.html"""
        if "+" in t:
            return t[t.index("+")+1:]
        else:
            return t

    tags = [simplify_tag(tag) for (word,tag) in corpus.tagged_words()]
    fd_tags =  nltk.FreqDist(tags)
    for tag in list(fd_tags.keys())[:n]:
        print(tag)
    del fd_tags
    del tags

In [ ]:
n = 10
print(f"As {n} tags mais comuns em pt-BR:")
most_commom_tags(mac_morpho, n)


In [ ]:
fd = nltk.FreqDist(mac_morpho.tagged_words())

In [ ]:
n = 10
print(f"As {n} palavras mais comuns em pt-BR:\n{fd.most_common(n)}")
del fd

# [PARTE 3] Testando modelos de determinação de Tags com o nltk

ESCREVER ALGO AQUI

## Dados de Treino e Dados de Teste

Vamos usar 80% dos dados para testes e 20% para treino.

In [7]:
k = 0.8
data = mac_morpho.tagged_sents()
tot = len(data)
tot_train_samples = int(np.ceil(tot*k))

train_data = data[:tot_train_samples]
test_data = data[tot_train_samples:]

del data, k, tot, tot_train_samples

## Fábricas de Modelos

Para facilitar o desenvolvimento do trabalho, será feito uma fábrica de modelos para criação e gerenciamento dos mesmos.
Abaixo explicamos a função de cada classe criada.

### Tipos de Modelos

Aqui definimos os tipos de modelos que serão usados para avaliar a acurácia.

In [69]:
class ModelType:
    TAGGER = 'TAGGER'
    AFFIX2 = 'AFFIX2'
    AFFIX3= 'AFFIX3'
    AFFIX4 = 'AFFIX4'
    AFFIX5 = 'AFFIX5'
    AFFIX6 = 'AFFIX6'
    UNIGRAM = 'UNIGRAM'
    BIGRAM = 'BIGRAM'
    TRIGRAM = 'TRIGRAM'
    BRILL_TAGGER = 'BRILL_TAGGER'
    NAIVES_BAYES = 'NAIVES_BAYES'

### Tagger de Modelos

Uma classe básica que contém informações dos modelos (valor padrão no texto, modelos, classificador usado,
acurácia e se a acurácia está em porcentual ou em valores unitário). Foi feito uma subclasse para
controle dos modelos.

In [78]:
class Tagger:
    class ModelDict:
        def __init__(self, name: str, classifier):
            self._data = {
                'name': name,
                'accuracy': None,
                'accuracy_type': "unit",
                'model': None,
                'classifier': classifier
            }

        @property
        def name(self) -> str:
            return self['name']

        @property
        def accuracy(self) -> Union[float, None]:
            return self['accuracy']

        @property
        def model(self):
            return self['model']

        @property
        def classifier(self):
            return self['classifier']

        def __str__(self):
            return str(self._data)

        def __getitem__(self, item: str):
            return self._data[item]

        def __setitem__(self, item: str, value):
            self._data[item] = value

        __repr__ = __str__

    def __init__(self, value: str):
        self._data = {
            "value": value,
            'models': {
                ModelType.TAGGER: Tagger.ModelDict(ModelType.TAGGER, nltk.DefaultTagger),
                ModelType.AFFIX2: Tagger.ModelDict(ModelType.AFFIX2, nltk.AffixTagger),
                ModelType.AFFIX3: Tagger.ModelDict(ModelType.AFFIX3, nltk.AffixTagger),
                ModelType.AFFIX4: Tagger.ModelDict(ModelType.AFFIX4, nltk.AffixTagger),
                ModelType.AFFIX5: Tagger.ModelDict(ModelType.AFFIX5, nltk.AffixTagger),
                ModelType.AFFIX6: Tagger.ModelDict(ModelType.AFFIX6, nltk.AffixTagger),
                ModelType.UNIGRAM: Tagger.ModelDict(ModelType.UNIGRAM, nltk.UnigramTagger),
                ModelType.BIGRAM: Tagger.ModelDict(ModelType.BIGRAM, nltk.BigramTagger),
                ModelType.TRIGRAM: Tagger.ModelDict(ModelType.TRIGRAM, nltk.TrigramTagger),
                ModelType.BRILL_TAGGER: Tagger.ModelDict(ModelType.BRILL_TAGGER, nltk.BrillTaggerTrainer),
                ModelType.NAIVES_BAYES: Tagger.ModelDict(ModelType.NAIVES_BAYES, nltk.ClassifierBasedPOSTagger)
    }
        }

    @property
    def value(self) -> str:
        return self['value']

    @property
    def models(self)->dict:
        return self['models']


    def evaluate_models(self, test_data, name):
        with tqdm.notebook.tqdm(self.models, position=1, desc=f'Evaluating models for {name}') as inner_for:
            for m in inner_for:
                model = self.models[m]
                accuracy_model = model.accuracy
                if accuracy_model is not None and isinstance(accuracy_model, float):
                    pass
                m_model = model.evaluate(test_data)
                self['models'][m]['accuracy'] = m_model


    def generate_models(self, train_data, name):
        with tqdm.notebook.tqdm(self.models, position=1, desc=f'Generating models for {name}') as inner_for:
            for m in inner_for:
                current_model = self['models'][m]['model']
                pathlib.Path("models").mkdir(parents=True, exist_ok=True)
                file_name = f"POS_tagger_{self['value']}_{self['models'][m]['name']}"
                file_path = f"models/{file_name}.pkl"
                tagger = None
                if current_model is None:
                    try:
                        self['models'][m]['model'] = joblib.load(file_path)
                    except FileNotFoundError:
                        if m == ModelType.TAGGER:
                            tagger = self['models'][m]['classifier'](self['value'])
                        elif m == ModelType.NAIVES_BAYES:
                            tagger = self['models'][m]['classifier'](train=train_data)
                        elif m == ModelType.BRILL_TAGGER:
                            backoff = self['models'][ModelType.TRIGRAM]['model']
                            tagger = self['models'][m]['classifier'](backoff, nltk.brill.fntbl37(), trace=True)
                            tagger = tagger.train(train_data)
                        elif m == ModelType.AFFIX2:
                            backoff = self['models'][ModelType.TAGGER]['model']
                            tagger = self['models'][m]['classifier'](train_data,affix_length=-2, backoff=backoff)
                        elif m == ModelType.AFFIX3:
                            backoff = self['models'][ModelType.AFFIX2]['model']
                            tagger = self['models'][m]['classifier'](train_data,affix_length=-3, backoff=backoff)
                        elif m == ModelType.AFFIX4:
                            backoff = self['models'][ModelType.AFFIX3]['model']
                            tagger = self['models'][m]['classifier'](train_data,affix_length=-4, backoff=backoff)
                        elif m == ModelType.AFFIX5:
                            backoff = self['models'][ModelType.AFFIX4]['model']
                            tagger = self['models'][m]['classifier'](train_data,affix_length=-5, backoff=backoff)
                        elif m == ModelType.AFFIX6:
                            backoff = self['models'][ModelType.AFFIX5]['model']
                            tagger = self['models'][m]['classifier'](train_data,affix_length=-6, backoff=backoff)
                        elif m == ModelType.UNIGRAM:
                            backoff = self['models'][ModelType.AFFIX6]['model']
                            tagger = self['models'][m]['classifier'](train_data, backoff=backoff)
                        elif m == ModelType.BIGRAM:
                            backoff = self['models'][ModelType.UNIGRAM]['model']
                            tagger = self['models'][m]['classifier'](train_data, backoff=backoff)
                        elif m == ModelType.TRIGRAM:
                            backoff = self['models'][ModelType.BIGRAM]['model']
                            tagger = self['models'][m]['classifier'](train_data, backoff=backoff)
                        if tagger is not None:
                            self['models'][m]['model'] = tagger
                            joblib.dump(tagger, file_path)

    def __getitem__(self, item):
        return self._data[item]

    def __setitem__(self, item, value):
        self._data[item] = value

    def __str__(self):
        return str(self._data)

    __repr__ = __str__

### Dícionarios de Tags

Classe que contém todas as tags utilizadas e com possibilidade de gerar todas (somente algumas), além de avaliá-lo.

In [75]:
class TagsDict:
    def __init__(self, train_data, test_data):
        self._train_data = train_data
        self._test_data = test_data
        self._data = {
            "ADJETIVO": Tagger("ADJ"),
            "ADVÉRBIO": Tagger("ADV"),
            "ADVÉRBIO CONECTIVO SUBORDINATIVO": Tagger("ADV-KS"),
            "ADVÉRBIO RELATIVO SUBORDINATIVO": Tagger("ADV-KS-REL"),
            "ARTIGO": Tagger("ART"),
            "CONJUNÇÃO COORDENATIVA": Tagger("KC"),
            "CONJUNÇÃO SUBORDINATIVA": Tagger("KS"),
            "INTERJEIÇÃO": Tagger("IN"),
            "NOME": Tagger("N"),
            "NOME PRÓPRIO": Tagger("NPROP"),
            "NUMERAL": Tagger("NUM"),
            "PARTICÍPIO": Tagger("PCP"),
            "PALAVRA DENOTATIVA": Tagger("PDEN"),
            "PREPOSIÇÃO": Tagger("PREP"),
            "PRONOME ADJETIVO": Tagger("PROADJ"),
            "PRONOME CONECTIVO SUBORDINATIVO": Tagger("PRO-KS"),
            "PRONOME PESSOAL": Tagger("PROPESS"),
            "PRONOME RELATIVO CONECTIVO SUBORDINATIVO": Tagger("PRO-KS-REL"),
            "PRONOME SUBSTANTIVO": Tagger("PROSUB"),
            "VERBO": Tagger("V"),
            "VERBO AUXILIAR": Tagger("VAUX"),
            "SÍMBOLO DE MOEDA CORRENTE": Tagger("CUR")
        }

    def train_all_models(self):
        with tqdm.notebook.tqdm(self._data, position=0, desc="Model Generating...") as main_for:
            for t in main_for:
                self._data[t].generate_models(self._train_data, t)

    def evaluate_all_models(self):
        with tqdm.notebook.tqdm(self._data, position=0, desc="Model Evaluating...") as main_for:
            for t in main_for:
                self._data[t].evaluate_models(self._test_data, t)

    def __str__(self):
        return str(self._data)
    __repr__ = __str__

## Conhecendo os classificadores do NLTK

Aqui faremos uma breve explicação de como cada classificador do NLTK funciona para melhor entendimento.

### Default Tagger

Primeiramente iremos utilizar o Default Tagger do nltk para classificar o texto aleatoriamente em alguma
classe gramatical. Ela servirá como um valor base para demais métodos que serão usados. Como padrão, iremos
usar a tag N, de nome. A escolha é puramente por ser a tag mais comum, mas qualquer outra tag poderia
ser usada.


### Affix

O AffixTagger se baseia em sufixos e prefixos. A língua portuguesa tem uma forma relação
entre os sufixos das palavras e sua classe. À exemplo: palavras terminar em -er, -ir ou -ar são em
sua maioria verbos. Palavras terminadas em -mente são advérbios de modo. O limite colocado foi 6,
justamente pois -mente é o maior sufixo existente em portugues, portando de 5 para 6 não haverá grandes
diferenças. Os prefixos foram ignorados por não apresentarem muita relação.

### Unigram
O UnigramTagger considera cada palavra de uma vez e determina o contexto da palavra.

### Bigram
O BigramTagger funciona como o Unigram, mas considera o conjunto de duas palavras: "Eu amo batata", os bigramas
são "eu amo" e "amo batata"

### Trigram
O TrigramTagger segue a mesma lógica do Unigram e do Trigram.


### Brill Tagger
O Brill Tagger é um tipo de classificador de aprendizado supervisionado que tenta determinar regras
para ajudar na classificação, i.e, o erro é minimizado na classificação seguida de associar uma tag a uma regra
gerada pelo algoritmo.

## Gerando os modelos

Aqui iremos gerar todos os modelos para futura avaliação.

In [ ]:
tags_dict = TagsDict(test_data=test_data, train_data=train_data)
tags_dict.train_all_models()

Model Generating...:   0%|          | 0/22 [00:00<?, ?it/s]

Generating models for ADJETIVO:   0%|          | 0/11 [00:00<?, ?it/s]

Generating models for ADVÉRBIO:   0%|          | 0/11 [00:00<?, ?it/s]

TBL train (fast) (seqs: 41118; tokens: 913108; tpls: 37; min score: 2; min acc: None)
Finding initial useful rules...
    Found 848255 useful rules.
Selecting rules...
TBL train (fast) (seqs: 41118; tokens: 913108; tpls: 37; min score: 2; min acc: None)
Finding initial useful rules...
    Found 848255 useful rules.
Selecting rules...


Generating models for ADVÉRBIO CONECTIVO SUBORDINATIVO:   0%|          | 0/11 [00:00<?, ?it/s]

## Acuracias

Aqui iremos avaliar a acurácias de todos os modelos por meio do métodos .evalute() de todos os classificadores
do nltk. Em seguida, iremos compará-los.


In [ ]:
tags_dict.evaluate_all_models()